In [ ]:
%env AWS_REQUEST_PAYER=requester

In [ ]:
from stacLib.ard_stac_search import Astac
import rasterio as rio
from rasterio.windows import Window

import numpy as np
import tqdm.notebook as tq
import zarr

In [ ]:
my_date='2020-01-01/2024-12-31'

In [ ]:
stac_config = {}

stac_config['url'] = 'https://landsatlook.usgs.gov/stac-server'
stac_config['fav_collection'] = 'landsat-c2l3-dswe'
stac_config['description'] = 'Landsat Collection 2 ARD Tiles DSWE'


In [ ]:
H='21'
V='10'
YEAR=2021

In [ ]:
chunkY = 250
chunkX = 250

In [ ]:
_stac = Astac(stac_config)

In [ ]:
date_range_text = my_date
cloud_cover_pct_max = '100'
_assets = _stac.get_my_assets(None, H, V, date_range_text, cloud_cover_pct_max)

In [ ]:
_assets[0]['properties']

In [ ]:

def read_tif(path: str, band: int = 1, window: Window = None):
    return rio.open(path).read(band, window=window)

def read_ls_data(item, band, window):
    return read_tif(item['assets'][band]['alternate']['s3']['href'], window=window)

In [ ]:
len(_assets)

In [ ]:
# Initialize 3d array time Y X

In [ ]:
window = Window(0, 0, chunkY, chunkX)

In [ ]:
big3d_ary = np.zeros(shape=(len(_assets), chunkY, chunkX), dtype=np.uint16)

In [ ]:
big3d_ary.shape

In [ ]:
%%time
p_bar=tq.tqdm(range(len(_assets)), position=0, leave=True)
myband='intsm'
for t1 in range(len(_assets)):
    item = _assets[t1]
    big3d_ary[t1] = read_ls_data(item, myband, window)
    p_bar.update(1)
    p_bar.refresh()

In [ ]:
Y=0
X=0
Z_store = f's3://ws-out/dwse/_H{H}V{V}_Y{Y}X{X}_store.zarr'

In [ ]:
%%time
# Save the 3D NumPy array as a Zarr array
zarr.save(Z_store, big3d_ary)

# To load the Zarr array back into a NumPy array
loaded_array = zarr.load(Z_store)

# Verify the loaded array
print(loaded_array.shape)

In [ ]:
# for Y in range(20):
#     for X in range(20):
#         work = f'H{H}V{V}_Y{Y}X{X}'
#         print(work)

In [ ]:
file_name='dancecard.txt'
mode='w'
f = open(file_name, mode)

In [ ]:
for Y in range(20):
    for X in range(20):
        work = f'H{H}V{V}_Y{Y}X{X}\n'
        f.write(work)

f.close()

In [ ]:
! cat dancecard.txt